In [12]:
import os
os.chdir('F:/datasets')


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [4]:
df=pd.read_csv('titanic.csv')

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Dropping of columns

In [17]:
df=df.drop(columns=['PassengerId','Name','Ticket','Cabin'])

KeyError: "['PassengerId', 'Name', 'Ticket', 'Cabin'] not found in axis"

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=1)

In [8]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
301,3,male,NaN,2,0,23.2500,Q
309,1,female,30.0,0,0,56.9292,C
516,2,female,34.0,0,0,10.5000,S
120,2,male,21.0,2,0,73.5000,S
570,2,male,62.0,0,0,10.5000,S
...,...,...,...,...,...,...,...
715,3,male,19.0,0,0,7.6500,S
767,3,female,30.5,0,0,7.7500,Q
72,2,male,21.0,0,0,73.5000,S
235,3,female,NaN,0,0,7.5500,S


#### Imputation Transformer

In [28]:
# All changes are done in the original df only and no other numpy array is creater

In [24]:
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]), #index value of age column is written 
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])#index of embarked column
],remainder='passthrough')#without passthrough all the other columns would be dropped

#### OHE

In [27]:
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_o utput=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

#### Scaling

In [30]:
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

#### Feature selection

In [32]:
trf4=SelectKBest(score_func=chi2,k=8)# top 8 most important columns to be used

#### Train the model

In [33]:
trf5=DecisionTreeClassifier ()

### Create Pipeline

In [54]:
pipe=Pipeline([  #list of tuples to be passes
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

### Pipeline vs make_pipeline

In [36]:
#Pipeline requires naming of steps, make_pipeline does not.
#(Same applies to ColumnTransformer vs make_column_transformer)

#### Alternate system

In [60]:
# pipe=make_pipeline(trf1,trf2,trf3,trf4)

In [55]:
#train
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001F1A0D5E7A0>)),
                ('trf5', DecisionTreeClassifier())])

#### Explore the pipeline

In [56]:
pipe.named_steps #pipe likh ke . like ke tab press karo

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x000001F1A0D5E7A0>),
 'trf5': DecisionTreeClassifier()}

In [67]:
 pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([30.16623239])

In [42]:
# Display pipeline

from sklearn import set_config
set_config(display='diagram')

In [68]:
#Predict
y_pred=pipe.predict(X_test)


In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6201117318435754

### Cross Validation using Pipeline

In [74]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()
# here we have done train_test_split for 5 times(cv=5),run our algo every time and then calculates the mean score 

0.6404510981975771

### GridSearch using Pipeline

In [75]:
params={
    'trf5__max_depth':[1,2,3,4,5,None]
}# hyper paremeter tuning ,kisi bhi algo me uske settings ko change karke  performanece ko improve kar sakte ho

In [78]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)
grid.best_score_

0.6404510981975771

In [79]:
grid.best_params_ # this shows that max depth should be 4

{'trf5__max_depth': 4}

#### Exporting the pipeline

In [80]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))
